# Todo 
1. Load the dataset for 1 date, explore the different metrics (strike distribution, number of calls vs puts, expiry available, forward curve, etc)
2. For a given several maturities 𝑇_1,𝑇_2,𝑒𝑡𝑐, plot the implied volatility with respect to strike 𝐾, moneyness 𝑚, delta Δ for both calls and puts.
3. For a chosen maturity 𝑇_1, and using the associated 𝑆_𝑡,𝐹_𝑡,𝜎_(𝑚𝑘𝑡,𝑖) calibrate a SABR model. More precisely use Hagan’s formula (2002) and minimize the $SSE=\sum_{option\ i}\left(\sigma_{mkt,i}-\sigma_{SABR,i}\right)$ or MSE using scipy.minimize. You can use a L-BFGS-B solver.
     - Generalize to all the other smiles of this single day.
     - Using the tools provided compute the Black & Scholes (1973) greeks and prices, compare with initial values.
4. Repeat 3. but for an SSVI model Gatheral & Jacquier (2014) to get all the surface at once.
5. Generalize 3 or 4 to multiple days, compute the realized volatility for different windows.
6. Using the rule-based selection function explore the implied volatility structure through time and strikes, vs realized, skew etc.


In [1]:
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc("font", **{"size": 22})

from investment_lab.option_selection import select_options
from investment_lab.metrics.volatility import rolling_realized_volatility
from investment_lab.metrics.util import levels_to_returns
from investment_lab.data.option_db import OptionLoader, extract_spot_from_options
from investment_lab.data.rates_db import USRatesLoader
from investment_lab.pricing.black_scholes import black_scholes_price, black_scholes_greeks
from investment_lab.pricing.implied_volatility import implied_volatility_vectorized
from investment_lab.rates import compute_forward
from investment_lab.metrics.distance import mse


In [2]:
df_options = OptionLoader.load_data(datetime(2022, 1, 4), datetime(2022, 1, 4), process_kwargs={"ticker":"SPY"},)
df_options.head()

2026-01-22 23:24:02,645 | INFO | Reading between 2022-01-04 00:00:00 2022-01-04 00:00:00 from ..//data/optiondb_2016_2023.parquet


,spot,strike,expiration,delta,gamma,vega,theta,rho,implied_volatility,bid,ask,volume,call_put,option_id,mid,ticker,date,day_to_expiration,moneyness
185309,477.56,300.0,2022-01-05,0.90942,0.00005,0.03701,-0.02497,0.00172,4.32829,177.31,177.86,0.0,C,SPY 20220105C300,177.585,SPY,2022-01-04,1,0.628193
185310,477.56,305.0,2022-01-05,0.91136,0.00003,0.03717,-0.02516,0.00169,4.13848,172.31,172.86,0.0,C,SPY 20220105C305,172.585,SPY,2022-01-04,1,0.638663
185311,477.56,310.0,2022-01-05,0.91280,0.00001,0.03752,-0.02504,0.00183,3.95528,167.31,167.86,0.0,C,SPY 20220105C310,167.585,SPY,2022-01-04,1,0.649133
185312,477.56,315.0,2022-01-05,0.91498,0.00002,0.03728,-0.01544,0.00176,3.77503,162.34,162.81,0.0,C,SPY 20220105C315,162.575,SPY,2022-01-04,1,0.659603
185313,477.56,320.0,2022-01-05,0.91630,0.00001,0.03703,-0.01475,0.00102,3.60230,157.34,157.81,0.0,C,SPY 20220105C320,157.575,SPY,2022-01-04,1,0.670073


In [3]:
df_spot = extract_spot_from_options(df_options)
df_spot.head()

,date,spot
0,2022-01-04,477.56


In [4]:
df_rates = USRatesLoader.load_data(datetime(2022, 1, 4), datetime(2022, 1, 4))
df_rates.head()

2026-01-22 23:24:25,191 | INFO | Reading between 2022-01-04 00:00:00 2022-01-04 00:00:00 from ..//data/par-yield-curve-rates-2020-2023.csv


,date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
503,2022-01-04,0.0006,0.0005,0.0008,NaN,0.0022,0.0038,0.0077,0.0102,0.0137,0.0157,0.0166,0.021,0.0207


In [5]:
df_options = compute_forward(df_options=df_options, df_rates=df_rates)
df_options.head()

,spot,strike,expiration,delta,gamma,vega,theta,rho,implied_volatility,bid,...,volume,call_put,option_id,mid,ticker,date,day_to_expiration,moneyness,risk_free_rate,forward
0,477.56,300.0,2022-01-05,0.90942,0.00005,0.03701,-0.02497,0.00172,4.32829,177.31,...,0.0,C,SPY 20220105C300,177.585,SPY,2022-01-04,1,0.628193,0.0006,477.560784
1,477.56,305.0,2022-01-05,0.91136,0.00003,0.03717,-0.02516,0.00169,4.13848,172.31,...,0.0,C,SPY 20220105C305,172.585,SPY,2022-01-04,1,0.638663,0.0006,477.560784
2,477.56,310.0,2022-01-05,0.91280,0.00001,0.03752,-0.02504,0.00183,3.95528,167.31,...,0.0,C,SPY 20220105C310,167.585,SPY,2022-01-04,1,0.649133,0.0006,477.560784
3,477.56,315.0,2022-01-05,0.91498,0.00002,0.03728,-0.01544,0.00176,3.77503,162.34,...,0.0,C,SPY 20220105C315,162.575,SPY,2022-01-04,1,0.659603,0.0006,477.560784
4,477.56,320.0,2022-01-05,0.91630,0.00001,0.03703,-0.01475,0.00102,3.60230,157.34,...,0.0,C,SPY 20220105C320,157.575,SPY,2022-01-04,1,0.670073,0.0006,477.560784
